In [31]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier 
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sbn
%matplotlib inline


def team_to_int_dict():
    teams = list(Game.objects.all().values_list('team', flat=True).distinct())
    return {team:teams.index(team) for team in teams}

def prep_model_input(player_list):
    player_stats =  np.array([[team_ids[stat.game.team], 
                               stat.game.home] 
                               for stat in player_list])
    player_names = [stat.player.name for stat in player_list]
    return player_stats, player_names


def create_predictions(train_input, train_answer, test_input):
    rfc = RandomForestClassifier ()
    rfc.fit(train_input, train_answer)
    prediction_float = rfc.predict(test_input)
    return [int(num) for num in prediction_float]
    

def plot_pred_actual(player_stat, predicted, test_answer, player_labels):
    plt.figure(figsize=(13,7))
    plt.title(player_stat)
    plt.plot(test_answer, 'g', label='Actual')
    plt.plot(predicted, 'r', label='Predicted')
    label_length = range(len(player_labels))
    plt.xticks(label_length, player_labels)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()


def get_train_test_players():
    train_players = PlayerData.objects.filter(game__week__lte=13)
    test_players = PlayerData.objects.filter(game__week=14)
    return train_players, test_players

def player_stat_arrays(player_list):
    final_dict = player_list[0].to_dict()
    final_dict = {stat_name:np.array([]) for stat_name in final_dict.keys()}
    for player in player_list:
        player_dict = player.to_dict()
        for stat in final_dict.keys():
            new_list = np.append(final_dict[stat], [player_dict[stat]])
            final_dict[stat] = new_list
    return final_dict

def predict_players():
    train_players, test_players = get_train_test_players()
    train_input, train_player_names = prep_model_input(train_players)
    test_input, test_player_names = prep_model_input(test_players)
    train_players_dict = player_stat_arrays(train_players)
    test_players_dict = player_stat_arrays(test_players)
    
    final_player_pred_dict = {name:{} for name in test_player_names}
    for player_stat in train_players_dict.keys():
        predicted = create_predictions(train_input, train_players_dict[player_stat], test_input)
        accuracy = accuracy_score(test_players_dict[player_stat], predicted)
        
        for name, prediction in zip(test_player_names, predicted):
            final_player_pred_dict[name][player_stat] = prediction
    return final_player_pred_dict
    
        # plot_pred_actual(player_stat, predicted, test_players_dict[player_stat], test_player_names)
        # print('{} accuracy score = {}'.format(player_stat, accuracy))

team_ids = team_to_int_dict()

In [35]:
new_dict = predict_players()

final_list =[]
for name, stats in new_dict.items():
    stats['name'] = name
    final_list.append(stats)
    
final_list

[{'interceptions': 2,
  'name': 'Andrew McNulty',
  'pass_attempts': 21,
  'pass_completions': 1,
  'pass_touchdowns': 1,
  'pass_yards': 101,
  'rec_touchdowns': 0,
  'rec_yards': 0,
  'receptions': 0,
  'rush_attempts': 13,
  'rush_touchdowns': 0,
  'rush_yards': 40},
 {'interceptions': 1,
  'name': 'Keenan Reynolds',
  'pass_attempts': 8,
  'pass_completions': 6,
  'pass_touchdowns': 0,
  'pass_yards': 30,
  'rec_touchdowns': 0,
  'rec_yards': 0,
  'receptions': 0,
  'rush_attempts': 21,
  'rush_touchdowns': 1,
  'rush_yards': 27},
 {'interceptions': 0,
  'name': 'Driphus Jackson',
  'pass_attempts': 24,
  'pass_completions': 11,
  'pass_touchdowns': 2,
  'pass_yards': 230,
  'rec_touchdowns': 0,
  'rec_yards': 0,
  'receptions': 0,
  'rush_attempts': 4,
  'rush_touchdowns': 0,
  'rush_yards': 29},
 {'interceptions': 0,
  'name': 'Reginald Bell Jr.',
  'pass_attempts': 16,
  'pass_completions': 0,
  'pass_touchdowns': 0,
  'pass_yards': 0,
  'rec_touchdowns': 0,
  'rec_yards': 0,
  

In [37]:
from collections import namedtuple
player_predict = namedtuple('player_predict', final_list[0].keys())
obj_list = [player_predict(**player) for player in final_list]

In [38]:
obj_list

[player_predict(name='Andrew McNulty', pass_yards=101, rush_attempts=13, rush_touchdowns=0, interceptions=2, receptions=0, rec_yards=0, pass_completions=1, rush_yards=40, rec_touchdowns=0, pass_touchdowns=1, pass_attempts=21),
 player_predict(name='Keenan Reynolds', pass_yards=30, rush_attempts=21, rush_touchdowns=1, interceptions=1, receptions=0, rec_yards=0, pass_completions=6, rush_yards=27, rec_touchdowns=0, pass_touchdowns=0, pass_attempts=8),
 player_predict(name='Driphus Jackson', pass_yards=230, rush_attempts=4, rush_touchdowns=0, interceptions=0, receptions=0, rec_yards=0, pass_completions=11, rush_yards=29, rec_touchdowns=0, pass_touchdowns=2, pass_attempts=24),
 player_predict(name='Reginald Bell Jr.', pass_yards=0, rush_attempts=2, rush_touchdowns=0, interceptions=0, receptions=0, rec_yards=0, pass_completions=0, rush_yards=202, rec_touchdowns=0, pass_touchdowns=0, pass_attempts=16),
 player_predict(name='Gary Nova', pass_yards=156, rush_attempts=2, rush_touchdowns=0, inter